In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv('train.csv',encoding="utf-8")
test=pd.read_csv('test.csv',encoding="utf-8")

In [17]:
df = pd.DataFrame(train["game_id"].unique(),columns=["game_id"])

In [18]:
df_t = pd.DataFrame(test["game_id"].unique(),columns=["game_id"])

### Ability만 분류

In [6]:
df_train = train[train["event"] == "Ability"]
df_test = test[test["event"] == "Ability"]

In [7]:
# 칼럼 단어만 추출하기
def eliminated_sc(text):
    word = text.split(';')
    col_name = word[0].split(' - ')
    if (len(col_name) >1):
        result = col_name[1]
    else:
        result = 'nothing'
    return result

In [8]:
df_train.event_contents = df_train.event_contents.apply(lambda x : eliminated_sc(x))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df_test.event_contents = df_test.event_contents.apply(lambda x : eliminated_sc(x))

In [8]:
df_train[:30]

,game_id,winner,time,player,species,event,event_contents
3,0,1,0.02,0,T,Ability,TrainSCV
19,0,1,0.27,0,T,Ability,BuildSupplyDepot
27,0,1,0.31,0,T,Ability,Stop
29,0,1,0.33,1,T,Ability,TrainSCV
37,0,1,0.35,0,T,Ability,TrainSCV
43,0,1,0.37,1,T,Ability,BuildSupplyDepot
56,0,1,0.40,1,T,Ability,BuildRefinery
63,0,1,0.41,1,T,Ability,TrainSCV
87,0,1,1.00,0,T,Ability,BuildBarracks
101,0,1,1.03,0,T,Ability,TrainSCV


In [10]:
importance_building = ['BuildBarracks','BuildExtractor','BuildGateway','BuildRefinery','BuildAssimilator','BuildHatchery',
                      'BuildCommandCenter','BuildNexus','BuildSpawningPool']
attack_building = ['BuildBarracks','BuildGateway','BuildSpawningPool']
gas_building = ['BuildExtractor','BuildRefinery','BuildAssimilator']
multi_building = ['BuildHatchery','BuildCommandCenter','BuildNexus']

In [21]:
def make_first_column(game_id,player,time,event_contents):
    if event_contents in importance_building:
        
        if event_contents in attack_building:
            event_contents = "BuildAttack"
        elif event_contents in gas_building:
            event_contents = "BuildGas"
        elif event_contents in multi_building:
            event_contents = "BuildMulti"
        
        player_num = "p"+str(player)
        col_name = player_num+"_first_"+str(event_contents)
        if col_name in df.columns:
            if(df.loc[game_id,col_name]>0):
                pass
            else:
                df.loc[game_id,col_name] = time
        else:
            df[col_name] = 0
            df.loc[game_id,col_name] = time

In [22]:
def make_first_column_test(game_id,player,time,event_contents):
    game_id = game_id - 38872
    if event_contents in importance_building:
        
        if event_contents in attack_building:
            event_contents = "BuildAttack"
        elif event_contents in gas_building:
            event_contents = "BuildGas"
        elif event_contents in multi_building:
            event_contents = "BuildMulti"
        
        player_num = "p"+str(player)
        col_name = player_num+"_first_"+str(event_contents)
        if col_name in df_t.columns:
            if(df_t.loc[game_id,col_name]>0):
                pass
            else:
                df_t.loc[game_id,col_name] = time
        else:
            df_t[col_name] = 0
            df_t.loc[game_id,col_name] = time

In [23]:
df_train.apply(lambda x : make_first_column(x["game_id"],x["player"],x["time"], x["event_contents"]) , axis = 1 )

3           None
19          None
27          None
29          None
37          None
            ... 
67091714    None
67091715    None
67091726    None
67091731    None
67091775    None
Length: 3798390, dtype: object

In [24]:
df_test.apply(lambda x : make_first_column_test(x["game_id"],x["player"],x["time"], x["event_contents"]) , axis = 1 )

3           None
29          None
69          None
75          None
82          None
            ... 
28714832    None
28714834    None
28714836    None
28714837    None
28714839    None
Length: 1631055, dtype: object

In [26]:
df

,game_id,p1_first_BuildGas,p0_first_BuildAttack,p0_first_BuildGas,p1_first_BuildAttack,p0_first_BuildMulti,p1_first_BuildMulti
0,0,0.40,1.00,1.09,1.21,2.14,0.00
1,1,1.56,0.53,1.08,0.58,7.36,2.55
2,2,6.19,0.54,1.08,1.50,6.45,3.30
3,3,1.00,0.56,1.03,0.54,2.27,9.26
4,4,5.19,0.58,1.31,2.13,2.28,1.19
...,...,...,...,...,...,...,...
38867,38867,0.42,1.06,0.42,0.00,2.34,6.49
38868,38868,3.17,1.04,1.41,3.56,9.48,2.33
38869,38869,0.19,1.58,1.26,0.00,4.06,3.31
38870,38870,1.00,1.51,0.42,0.53,4.09,2.00


In [18]:
df.to_csv("inv_ft_build_train.csv",encoding="utf-8",index=False)

In [25]:
df_t

,game_id,p0_first_BuildGas,p1_first_BuildGas,p1_first_BuildAttack,p1_first_BuildMulti,p0_first_BuildAttack,p0_first_BuildMulti
0,38872,1.11,1.31,4.05,4.41,4.51,0.00
1,38873,0.00,0.00,2.01,1.46,0.00,1.16
2,38874,0.45,0.41,0.00,0.00,1.05,0.00
3,38875,0.40,0.00,0.53,2.02,1.03,0.00
4,38876,0.26,0.20,1.27,0.00,0.00,0.00
...,...,...,...,...,...,...,...
16782,55654,1.49,0.40,1.04,2.37,1.50,1.17
16783,55655,0.28,0.40,1.04,3.30,1.51,7.43
16784,55656,1.09,5.05,4.04,0.00,0.00,0.00
16785,55657,0.00,0.46,1.02,0.00,0.57,0.00


In [19]:
df_t.to_csv("inv_ft_build_test.csv",encoding="utf-8",index=False)

In [3]:
df = pd.read_csv("ft_build_train.csv",encoding="utf-8")
df_t = pd.read_csv("ft_build_test.csv",encoding="utf-8")

In [5]:
game_id = df["game_id"]

In [13]:
test_game_id = df_t["game_id"]

In [6]:
df = df.apply(lambda x : (1/x))

In [14]:
df_t = df_t.apply(lambda x : (1/x))

In [8]:
df["game_id"] = game_id

In [15]:
df_t["game_id"] = test_game_id

In [9]:
df

,game_id,p1_first_BuildGas,p0_first_BuildAttack,p0_first_BuildGas,p1_first_BuildAttack,p0_first_BuildMulti,p1_first_BuildMulti
0,0,2.500000,1.000000,0.917431,0.826446,0.467290,inf
1,1,0.641026,1.886792,0.925926,1.724138,0.135870,0.392157
2,2,0.161551,1.851852,0.925926,0.666667,0.155039,0.303030
3,3,1.000000,1.785714,0.970874,1.851852,0.440529,0.107991
4,4,0.192678,1.724138,0.763359,0.469484,0.438596,0.840336
5,5,0.403226,0.763359,0.232019,1.818182,0.847458,0.120192
6,6,0.980392,1.694915,0.769231,inf,inf,inf
7,7,0.934579,inf,5.000000,1.851852,inf,0.500000
8,8,2.941176,1.000000,0.432900,0.500000,0.240385,0.746269
9,9,0.729927,1.694915,0.909091,inf,inf,0.877193


In [11]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [16]:
df_t = df_t.replace([np.inf, -np.inf], np.nan)
df_t = df_t.fillna(0)

In [12]:
df

,game_id,p1_first_BuildGas,p0_first_BuildAttack,p0_first_BuildGas,p1_first_BuildAttack,p0_first_BuildMulti,p1_first_BuildMulti
0,0,2.500000,1.000000,0.917431,0.826446,0.467290,0.000000
1,1,0.641026,1.886792,0.925926,1.724138,0.135870,0.392157
2,2,0.161551,1.851852,0.925926,0.666667,0.155039,0.303030
3,3,1.000000,1.785714,0.970874,1.851852,0.440529,0.107991
4,4,0.192678,1.724138,0.763359,0.469484,0.438596,0.840336
5,5,0.403226,0.763359,0.232019,1.818182,0.847458,0.120192
6,6,0.980392,1.694915,0.769231,0.000000,0.000000,0.000000
7,7,0.934579,0.000000,5.000000,1.851852,0.000000,0.500000
8,8,2.941176,1.000000,0.432900,0.500000,0.240385,0.746269
9,9,0.729927,1.694915,0.909091,0.000000,0.000000,0.877193


In [17]:
df_t

,game_id,p0_first_BuildGas,p1_first_BuildGas,p1_first_BuildAttack,p1_first_BuildMulti,p0_first_BuildAttack,p0_first_BuildMulti
0,38872,0.900901,0.763359,0.246914,0.226757,0.221729,0.000000
1,38873,0.000000,0.000000,0.497512,0.684932,0.000000,0.862069
2,38874,2.222222,2.439024,0.000000,0.000000,0.952381,0.000000
3,38875,2.500000,0.000000,1.886792,0.495050,0.970874,0.000000
4,38876,3.846154,5.000000,0.787402,0.000000,0.000000,0.000000
5,38877,2.380952,2.857143,0.961538,0.000000,0.952381,0.432900
6,38878,0.000000,0.729927,0.689655,0.000000,0.769231,0.000000
7,38879,0.970874,0.155280,2.127660,0.332226,1.785714,0.444444
8,38880,0.925926,0.497512,0.000000,0.869565,0.122399,0.125000
9,38881,0.970874,0.699301,0.000000,0.877193,1.785714,0.238663
